In [ ]:
#Use the following in a bash script
# apt install tesseract-ocr libtesseract-dev poppler-utils libgl1

In [ ]:
pip install -r ../requirements.in

In [19]:
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Title, NarrativeText, Text
from unstructured.chunking.basic import chunk_elements
from typing import List
import os
import weaviate
from weaviate.util import generate_uuid5
import ollama
from pprint import pprint
import json

In [ ]:
ollama.pull("mxbai-embed-large:v1")


In [27]:
ollama.list()

{'models': [{'name': 'mxbai-embed-large:v1',
   'model': 'mxbai-embed-large:v1',
   'modified_at': '2024-04-20T21:03:01.270699797Z',
   'size': 669615493,
   'digest': '468836162de7f81e041c43663fedbbba921dcea9b9fefea135685a39b2d83dd8',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'bert',
    'families': ['bert'],
    'parameter_size': '334M',
    'quantization_level': 'F16'}}]}

In [ ]:
# Use this for now
FILE_PATH = "../docs/Player_s Handbook.pdf"

elements = partition_pdf(filename=FILE_PATH)

In [ ]:
#save this for later
FILE_PATH = "../docs/Player_s Handbook.pdf"

elements = partition_pdf(filename=FILE_PATH, strategy="hi_res", infer_table_structure=True)

In [20]:
import weaviate
import weaviate.classes as wvc
client = weaviate.connect_to_local(
    additional_config=wvc.init.AdditionalConfig(
        timeout=(60,1800),
    ),
)
client.connect()

In [ ]:
import weaviate.classes as wvc
from weaviate.collections import Collection
from weaviate.client import WeaviateClient

def create_collection(client: WeaviateClient, collection_name: str)-> Collection:
    with client: 
        client.collections.delete(collection_name)
        client.collections.create(
            name=collection_name,
            vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_transformers(),
            vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
                distance_metric=wvc.config.VectorDistances.COSINE # select prefered distance metric
        ),
    )
    collection = client.collections.get(name=collection_name)
    return collection

collection = create_collection(client, 'PHB')


In [ ]:
narrative_texts = [elem for elem in elements if elem.category == "NarrativeText"]
titles = [elem for elem in elements if elem.category == "Title"]
uncat_texts = [elem for elem in elements if elem.category == "UncategorizedText"]


In [ ]:

for text in narrative_texts:
    pprint(text.to_dict())

In [ ]:
for title in titles:
    pprint(title.to_dict())

In [ ]:
for text in uncat_texts:
    pprint(text.to_dict())

In [ ]:
for elem in elements:
    pprint(elem.to_dict())

In [ ]:
element_dicts = [chunk.to_dict() for chunk in elements]



In [12]:
embeddings = []

for element in element_dicts:
    response = ollama.embeddings(model='mxbai-embed-large:v1', prompt=element['text'])
    embedding = response["embedding"]
    embeddings.append(embedding)
    

In [13]:
embeddings

NameError: name 'embeddings' is not defined

In [13]:
chunk_embeddings_with_metadata = [
        {
            "id":  None,
            "type": element['type'],
            "title": element['metadata']['filename'],
            "url": "None",
            "content": element['text'],
            "label": "No Label",
            "tokens": len(element['text'].split()),
            "embedding": embedding,
        }
        for element, embedding in zip(element_dicts, embeddings)
    ]

In [20]:
from typing import List, Dict 
from weaviate.client import WeaviateClient
import weaviate.classes as wvc
def load_chunks_into_weaviate(chunks: List[Dict], client: WeaviateClient, collection_name: str):
    collection = create_collection(client, collection_name)
    chunk_objs = []
    for chunk in chunks:
        chunk_obj = wvc.data.DataObject(
            properties={
                "content": chunk['content'],
                "tokens": chunk['tokens'],
                "title": chunk['title'],
                "type": chunk['type'],
                "url": chunk['url'],
                "label": chunk['label']
            }
        )    
        chunk_objs.append(chunk_obj)
        
    with client:
        collection.data.insert_many(chunk_objs)
        
    print(f"Loaded {len(chunks)} chunks into Weaviate")
        
load_chunks_into_weaviate(chunk_embeddings_with_metadata, client, 'PHB')

Loaded 13847 chunks into Weaviate


In [79]:
import weaviate.classes as wvc
from weaviate.collections import Collection
from weaviate.client import WeaviateClient
from pprint import pprint
def get_collection(client: WeaviateClient, collection_name: str)-> Collection:
    collection = client.collections.get(collection_name)
    return collection

client.connect()
test_collection = get_collection(client, "PHB")
pprint(test_collection)


In [208]:
from pprint import pprint
import ollama
prompt = "You are a Dungeons & Dragons assistant for version 3.5. Your frame of reference is in the provided document only."
test_query = "How far away can I cast a Fireball spell?"

test_response = ollama.embeddings(
    prompt=test_query,
    model='mxbai-embed-large:v1'
)
pprint(test_response["embedding"])

[-0.4379090368747711,
 -0.11808867752552032,
 -0.2770307660102844,
 -0.12051962316036224,
 0.5631095170974731,
 -0.10765093564987183,
 0.5703073143959045,
 0.3209840953350067,
 0.36567020416259766,
 0.884597897529602,
 1.8048548698425293,
 -0.34747275710105896,
 1.0071630477905273,
 -1.050893783569336,
 -0.4822838306427002,
 -0.1279420554637909,
 -1.2358354330062866,
 0.014009550213813782,
 -1.0183160305023193,
 -0.1565895974636078,
 -0.24075385928153992,
 0.26966702938079834,
 -0.9871354699134827,
 -1.2719122171401978,
 -0.6513684391975403,
 0.6693729162216187,
 0.531995415687561,
 -0.007818154990673065,
 0.7283023595809937,
 0.7623134851455688,
 -0.09291070699691772,
 -0.4317227900028229,
 -0.1483445167541504,
 -0.702239990234375,
 -0.5747934579849243,
 0.03717412054538727,
 0.2899174392223358,
 -0.4990341067314148,
 -0.17391207814216614,
 -0.47545337677001953,
 0.6301454901695251,
 -0.6648809313774109,
 1.2119545936584473,
 -0.6443039774894714,
 -0.11040698736906052,
 -0.76589876413

In [213]:
from pprint import pprint
test_results = test_collection.query.near_vector(
    near_vector=test_response["embedding"],
)
pprint(test_results)

QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('9c31d3b2-378c-402d-8fb9-462f0d81b904'),
                            metadata=MetadataReturn(creation_time=None,
                                                    last_update_time=None,
                                                    distance=None,
                                                    certainty=None,
                                                    score=None,
                                                    explain_score=None,
                                                    is_consistent=None,
                                                    rerank_score=None),
                            properties={'content': 'To cast a spell, you must '
                                                   'be able to speak (if the '
                                                   'spell has a verbal '
                                                   'component), gesture (if it '
                                  

In [210]:

from pprint import pprint
test_results_content = []
for obj in test_results.objects:
    test_results_content.append(obj.properties['content'])
pprint(test_results_content)

['To cast a spell, you must be able to speak (if the spell has a verbal '
 'component), gesture (if it has a somatic component), and manipu- late the '
 'material components or focus (if any). Additionally, you must concentrate to '
 'cast a spell—and it’s hard to concentrate in the head of battle. (See below '
 'for details.)',
 'As a spellcaster’s knowledge of magic grows, she can learn to cast spells in '
 'ways slightly different from the ways in which the spells were originally '
 'designed or learned. For example, a spellcaster can learn to cast a spell '
 'without having to say its verbal component, to cast a spell for greater '
 'effect, or even to cast it with nothing but a moment’s thought. Preparing '
 'and casting a spell in such a way is harder than normal but, thanks to '
 'metamagic feats, at least it is possible.',
 'You can cast a spell at a lower caster level than normal, but the caster '
 'level you choose must be high enough for you to cast the spell in question, '


In [ ]:
ollama.pull("mistral")


In [211]:
from pprint import pprint
test_prompt = f"Using this data: {test_results_content}, respond to this prompt: {test_query}"
pprint(test_prompt)


("Using this data: ['To cast a spell, you must be able to speak (if the spell "
 'has a verbal component), gesture (if it has a somatic component), and '
 'manipu- late the material components or focus (if any). Additionally, you '
 'must concentrate to cast a spell—and it’s hard to concentrate in the head of '
 "battle. (See below for details.)', 'As a spellcaster’s knowledge of magic "
 'grows, she can learn to cast spells in ways slightly different from the ways '
 'in which the spells were originally designed or learned. For example, a '
 'spellcaster can learn to cast a spell without having to say its verbal '
 'component, to cast a spell for greater effect, or even to cast it with '
 'nothing but a moment’s thought. Preparing and casting a spell in such a way '
 'is harder than normal but, thanks to metamagic feats, at least it is '
 "possible.', 'You can cast a spell at a lower caster level than normal, but "
 'the caster level you choose must be high enough for you to cast the 

In [212]:
from pprint import pprint
test_output = ollama.generate(
    model="mistral",
    prompt=test_prompt
)

pprint(test_output['response'])

(' The range for a Fireball spell is 15 feet plus 1 foot per caster level. '
 'Therefore, the exact distance you can cast a Fireball spell depends on your '
 'caster level. For example, a 1st-level wizard can cast Fireball up to 16 '
 'feet away (15 feet + 1 foot), while a 20th-level wizard can cast Fireball up '
 'to 71 feet away (15 feet + 1 foot per level up to 20, then an additional 1 '
 'foot for each additional level beyond 20).')
